In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import string
import re
from collections import Counter


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#remove punctuations
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def remove_stopwords(text, STOPWORDS):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def stem_words(text, stemmer):
    return " ".join([stemmer.stem(word) for word in text.split()])

def remove_freqwords(text, FREQWORDS):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

def lemmatize_words(text, lemmatizer, wordnet_map ):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

def clean_review(text):
    clean_text = []
    for w in word_tokenize(text):
        if w.lower() not in stop:
            pos = pos_tag([w])
            new_w = lemmatizer.lemmatize(w, pos=get_simple_pos(pos[0][1]))
            clean_text.append(new_w)
    return clean_text

def join_text(text):
    return " ".join(text)


In [33]:
#reading the data
training_set = pd.read_csv('drive/MyDrive/Text Analytics/training_set.txt',  encoding = "ISO-8859-1",header=None,sep=',',names = ["News","Sentiment"])
validation_set = pd.read_csv('drive/MyDrive/Text Analytics/validation_set.txt',  encoding = "ISO-8859-1",header=None,sep=',',names = ["News","Sentiment"])
test_set = pd.read_csv('drive/MyDrive/Text Analytics/test_set.txt',  encoding = "ISO-8859-1",header=None,sep=',',names = ["News","Sentiment"])
print(training_set.shape,validation_set.shape,test_set.shape)
print("COLUMN NAMES" , training_set.columns,validation_set.columns,test_set.columns)
print(training_set.info(),validation_set.info(),test_set.info())

(2265, 2) (1190, 2) (2581, 2)
COLUMN NAMES Index(['News', 'Sentiment'], dtype='object') Index(['News', 'Sentiment'], dtype='object') Index(['News', 'Sentiment'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Float64Index: 2265 entries, nan to 2263.0
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   News       2265 non-null   object
 1   Sentiment  2265 non-null   object
dtypes: object(2)
memory usage: 53.1+ KB
<class 'pandas.core.frame.DataFrame'>
Float64Index: 1190 entries, nan to 3447.0
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   News       1190 non-null   object
 1   Sentiment  1190 non-null   object
dtypes: object(2)
memory usage: 27.9+ KB
<class 'pandas.core.frame.DataFrame'>
Float64Index: 2581 entries, nan to 2579.0
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   News     

In [34]:
test_set = test_set[1:]
test_set.head()

,News,Sentiment
1.0,The international electronic industry company ...,negative
2.0,With the new production plant the company woul...,positive
3.0,According to the company 's updated strategy f...,positive
4.0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,positive
5.0,TeliaSonera TLSN said the offer is in line wit...,positive


In [35]:
#stopwords removal
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
print(stopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [36]:
#vettorizzazione del test 
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer( lowercase = True, strip_accents='ascii', stop_words = stopset)

In [37]:
training_set['News'] = training_set['News'].apply(lambda x: remove_punct(x))
", ".join(stopwords.words('english'))
validation_set['News'] = validation_set['News'].apply(lambda x: remove_punct(x))
", ".join(stopwords.words('english'))
test_set['News'] = test_set['News'].apply(lambda x: remove_punct(x))
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
training_set["text_wo_stop"] = training_set["News"].apply(lambda text: remove_stopwords(text, STOPWORDS))
validation_set["text_wo_stop"] = validation_set["News"].apply(lambda text: remove_stopwords(text, STOPWORDS))
test_set["text_wo_stop"] = test_set["News"].apply(lambda text: remove_stopwords(text, STOPWORDS))
validation_set.head(50)

,News,Sentiment,text_wo_stop
NaN,News,Sentiment,News
1.0,With the new production plant the company woul...,positive,With new production plant company would increa...
6.0,TeliaSonera TLSN said the offer is in line wit...,positive,TeliaSonera TLSN said offer line strategy incr...
7.0,STORA ENSO NORSKE SKOG MREAL UPMKYMMENE Cre...,positive,STORA ENSO NORSKE SKOG MREAL UPMKYMMENE Credit...
8.0,A purchase agreement for tons of gasoline wit...,positive,A purchase agreement tons gasoline delivery Ha...
14.0,Incap Contract Manufacturing Services Pvt Ltd ...,positive,Incap Contract Manufacturing Services Pvt Ltd ...
16.0,Lifetree was founded in and its revenues hav...,positive,Lifetree founded revenues risen average margin...
23.0,Operating profit for the ninemonth period incr...,positive,Operating profit ninemonth period increased EU...
31.0,Aspo Plc STOCK EXCHANGE RELEASE February at...,positive,Aspo Plc STOCK EXCHANGE RELEASE February ESL S...
36.0,Nyrstar has also agreed to supply to Talvivaar...,positive,Nyrstar also agreed supply Talvivaara tonnes s...


In [38]:
stemmer = PorterStemmer()
training_set["text_stemmed"] = training_set["text_wo_stop"].apply(lambda text: stem_words(text, stemmer))
validation_set["text_stemmed"] = validation_set["text_wo_stop"].apply(lambda text: stem_words(text, stemmer))
test_set["text_stemmed"] = test_set["text_wo_stop"].apply(lambda text: stem_words(text, stemmer))
training_set.head(2)

,News,Sentiment,text_wo_stop,text_stemmed
NaN,News,Sentiment,News,new
0.0,According to Gran the company has no plans to...,neutral,According Gran company plans move production R...,accord gran compani plan move product russia a...


In [39]:
#remove frequent words - countvectorization
cnt = Counter()
for text in training_set["text_stemmed"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(20)

[('eur', 954),
 ('the', 667),
 ('mn', 494),
 ('compani', 384),
 ('profit', 334),
 ('oper', 334),
 ('net', 292),
 ('sale', 278),
 ('million', 267),
 ('share', 264),
 ('finnish', 219),
 ('said', 206),
 ('mln', 198),
 ('period', 194),
 ('year', 183),
 ('quarter', 170),
 ('group', 160),
 ('euro', 155),
 ('total', 151),
 ('market', 137)]

In [40]:
#remove frequent words - countvectorization
cnt1 = Counter()
for text in validation_set["text_stemmed"].values:
    for word in text.split():
        cnt1[word] += 1
        
cnt1.most_common(20)

[('the', 353),
 ('compani', 259),
 ('eur', 220),
 ('said', 138),
 ('oper', 128),
 ('sale', 127),
 ('finnish', 124),
 ('million', 100),
 ('share', 93),
 ('new', 92),
 ('net', 86),
 ('finland', 85),
 ('servic', 84),
 ('year', 84),
 ('market', 81),
 ('group', 80),
 ('product', 75),
 ('mn', 74),
 ('busi', 72),
 ('profit', 67)]

In [41]:
#remove frequent words - countvectorization
cnt2 = Counter()
for text in test_set["text_stemmed"].values:
    for word in text.split():
        cnt2[word] += 1
        
cnt2.most_common(20)

[('the', 687),
 ('compani', 586),
 ('eur', 356),
 ('said', 337),
 ('finnish', 294),
 ('oper', 237),
 ('sale', 223),
 ('new', 207),
 ('finland', 205),
 ('servic', 204),
 ('product', 197),
 ('market', 180),
 ('million', 175),
 ('share', 172),
 ('group', 166),
 ('busi', 159),
 ('year', 148),
 ('develop', 141),
 ('also', 137),
 ('oyj', 124)]

In [42]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
training_set["text__stopfreq"] = training_set["text_stemmed"].apply(lambda text: remove_freqwords(text, FREQWORDS))
FREQWORDS = set([w for (w, wc) in cnt1.most_common(10)])
validation_set["text__stopfreq"] = validation_set["text_stemmed"].apply(lambda text: remove_freqwords(text, FREQWORDS))
FREQWORDS = set([w for (w, wc) in cnt2.most_common(10)])
test_set["text__stopfreq"] = test_set["text_stemmed"].apply(lambda text: remove_freqwords(text, FREQWORDS))

In [43]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
training_set["text_lemmatized"] = training_set["text__stopfreq"].apply(lambda text: lemmatize_words(text, lemmatizer, wordnet_map))
validation_set["text_lemmatized"] = validation_set["text__stopfreq"].apply(lambda text: lemmatize_words(text, lemmatizer, wordnet_map))
test_set["text_lemmatized"] = test_set["text__stopfreq"].apply(lambda text: lemmatize_words(text, lemmatizer, wordnet_map))
test_set.head(10)

,News,Sentiment,text_wo_stop,text_stemmed,text__stopfreq,text_lemmatized
1.0,The international electronic industry company ...,negative,The international electronic industry company ...,the intern electron industri compani elcoteq l...,intern electron industri elcoteq laid ten empl...,intern electron industri elcoteq lay ten emplo...
2.0,With the new production plant the company woul...,positive,With new production plant company would increa...,with new product plant compani would increas c...,with product plant would increas capac meet ex...,with product plant would increas capac meet ex...
3.0,According to the company s updated strategy fo...,positive,According company updated strategy years Baswa...,accord compani updat strategi year baswar targ...,accord updat strategi year baswar target longt...,accord updat strategi year baswar target longt...
4.0,FINANCING OF ASPOCOMP S GROWTH Aspocomp is agg...,positive,FINANCING OF ASPOCOMP S GROWTH Aspocomp aggres...,financ OF aspocomp S growth aspocomp aggress p...,financ OF aspocomp S growth aspocomp aggress p...,financ OF aspocomp S growth aspocomp aggress p...
5.0,TeliaSonera TLSN said the offer is in line wit...,positive,TeliaSonera TLSN said offer line strategy incr...,teliasonera tlsn said offer line strategi incr...,teliasonera tlsn offer line strategi increas o...,teliasonera tlsn offer line strategi increas o...
6.0,STORA ENSO NORSKE SKOG MREAL UPMKYMMENE Cre...,positive,STORA ENSO NORSKE SKOG MREAL UPMKYMMENE Credit...,stora enso norsk skog mreal upmkymmen credit s...,stora enso norsk skog mreal upmkymmen credit s...,stora enso norsk skog mreal upmkymmen credit s...
7.0,A purchase agreement for tons of gasoline wit...,positive,A purchase agreement tons gasoline delivery Ha...,A purchas agreement ton gasolin deliveri hamin...,A purchas agreement ton gasolin deliveri hamin...,A purchas agreement ton gasolin deliveri hamin...
8.0,Incap Contract Manufacturing Services Pvt Ltd ...,positive,Incap Contract Manufacturing Services Pvt Ltd ...,incap contract manufactur servic pvt ltd subsi...,incap contract manufactur pvt ltd subsidiari i...,incap contract manufactur pvt ltd subsidiari i...
9.0,Lifetree was founded in and its revenues hav...,positive,Lifetree founded revenues risen average margin...,lifetre found revenu risen averag margin late,lifetre found revenu risen averag margin late,lifetre find revenu rise averag margin late
10.0,Filippova A trilateral agreement on investme...,positive,Filippova A trilateral agreement investment co...,filippova A trilater agreement invest construc...,filippova A trilater agreement invest construc...,filippova A trilater agreement invest construc...


In [47]:
#training_set=training_set.drop(["text_stemmed","text__stopfreq"],axis=1)
training_set['encoded_category'] = LabelEncoder().fit_transform(training_set['Sentiment'])
#validation_set=validation_set.drop(["text_stemmed","text__stopfreq"],axis=1)
validation_set['encoded_category'] = LabelEncoder().fit_transform(validation_set['Sentiment'])
test_set=test_set.drop(["text_stemmed","text__stopfreq"],axis=1)
test_set['encoded_category'] = LabelEncoder().fit_transform(test_set['Sentiment'])


In [48]:
training_set=training_set.drop(["Sentiment","text_wo_stop","News"],axis=1)

In [60]:
training_set = training_set[1:]
training_set

,text_lemmatized,encoded_category
1.0,for last quarter componenta doubl eurm eurm pe...,3
2.0,In third quarter increas,3
3.0,rise correspond period repres,3
4.0,total repres,3
5.0,finnish talentum report increas total,3
...,...,...
2259.0,result month period decreas turnov decreas compar,1
2260.0,helsinki thomson financi cargotec fell sharpli...,1
2261.0,london marketwatch price end low london monday...,1
2262.0,fell includ vessel gain,1


In [54]:
#validation_set=validation_set.drop(["Sentiment","text_wo_stop","News"],axis=1)
validation_set = validation_set[1:]
validation_set.head()

,text_lemmatized,encoded_category
1.0,with product plant would increas capac meet ex...,3
6.0,teliasonera tlsn offer line strategi increas o...,3
7.0,stora enso norsk skog mreal upmkymmen credit s...,3
8.0,A purchas agreement ton gasolin deliveri hamin...,3
14.0,incap contract manufactur servic pvt ltd subsi...,3


In [56]:
test_set=test_set.drop(["Sentiment","text_wo_stop","News"],axis=1)
test_set.head()

,text_lemmatized,encoded_category
1.0,intern electron industri elcoteq lay ten emplo...,0
2.0,with product plant would increas capac meet ex...,2
3.0,accord updat strategi year baswar target longt...,2
4.0,financ OF aspocomp S growth aspocomp aggress p...,2
5.0,teliasonera tlsn offer line strategi increas o...,2


In [59]:
x_train,x_test,y_train,y_test = train_test_split(training_set.text_lemmatized,training_set.encoded_category,test_size = 0.1 , random_state = 0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2036,), (227,), (2036,), (227,))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(test_set.text_lemmatized,test_set.encoded_category,test_size = 0.3, random_state = 0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
pipe = Pipeline([('tfidf', TfidfVectorizer()),
                 ('model', LinearSVC())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("MODEL - LINEAR SVC")
print("accuracy score: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

In [ ]:
y = df.Sentiment

In [ ]:
#convert column news from text to features
x = vectorizer.fit_transform(df.News)

In [ ]:
print(y.shape)
print(x.shape)

(2264,)
(2264, 5945)


In [ ]:
#Test and Train Split
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42)

In [ ]:
clf = naive_bayes.MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [ ]:
roc_auc_score(y_test, clf.predict_proba(x_test)[:,1],multi_class='ovr')

AxisError: ignored